# Table of Contents<a name="toc"></a>
* [Implement PKCS\#7 padding](#prob9)
* [Implement CBC mode](#prob10)
* [An ECB/CBC detection oracle](#prob11)
* [Byte-at-a-time ECB decryption (Simple)](#prob12)
* [ECB cut-and-paste](#prob13)
* [Byte-at-a-time ECB decryption (Harder)](#prob14)
* [PKCS\#7 padding validation](#prob15)
* [CBC bitflipping attacks](#prob16)

In [5]:
%%capture
!pip install pycryptodome
!pip install numpy
from Crypto.Cipher import AES
import base64
import binascii
import numpy as np
import random
import uuid
from typing import Tuple, List
from enum import Enum, auto

# Implement PKCS\#7 padding<a name="prob9"></a>

In [19]:
def pkcs7(data: bytes, *, block_size: int = 16) -> bytes:
    padding = block_size - (len(data) % block_size)
    return data + bytes([padding for x in range(padding)])

In [17]:
pkcs7(b"YELLOW SUBMARINE", block_size=20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# Implement CBC mode<a name="prob10"></a>

<img src="img/CBC_decryption.svg">
<img src="img/CBC_encryption.svg">
So, if I understand AES CBC decryption correctly, the way it works is provided an initialization vector, a ciphertext block is deciphered and then XOR'd with the initialization vector to get the resulting plaintext. The next ciphertext block is decrypted  and XOR'd with the previous ciphertext block. CBC encryption works the same, only going the other way.

In [6]:
def xor_encrypt(cipher: bytes, block: bytes) -> bytes:
    cipher_npa = np.frombuffer(cipher, dtype=np.uint8)
    block_npa = np.frombuffer(block, dtype=np.uint8)
    return np.bitwise_xor(cipher_npa, block_npa).tobytes()
xor_decrypt = xor_encrypt

In [7]:
def aes_cbc_decipher(ciphertext: bytes, key: bytes, iv: bytes) -> bytes:
    decipher = AES.new(key, AES.MODE_ECB)
    plaintext = b""
    previous_block = iv
    while len(ciphertext) > 0:
        segment = ciphertext[:len(iv)]
        ciphertext = ciphertext[len(iv):]
        plaintext += xor_decrypt(decipher.decrypt(segment), previous_block)
        previous_block = segment
    return plaintext

In [8]:
def aes_cbc_encipher(plaintext: bytes, key: bytes, iv: bytes) -> bytes:
    encipher = AES.new(key, AES.MODE_ECB)
    ciphertext = b""
    previous_block = iv
    while len(plaintext) > 0:
        segment = plaintext[:len(iv)]
        plaintext = plaintext[len(iv):]
        encoded_block = encipher.encrypt(xor_encrypt(segment, previous_block))
        ciphertext += encoded_block
        previous_block = encoded_block
    return ciphertext

In [21]:
key = b"YELLOW SUBMARINE"
iv = b"\x00" * 16

with open("set2/10.txt") as fd:
    data = fd.read()
data = data.replace("\n", "")

ciphertext = base64.b64decode(data)
plaintext = aes_cbc_decipher(ciphertext, key, iv)

# check to see if encryptor works
encoded_text = aes_cbc_encipher(plaintext, key, iv)
assert(encoded_text == ciphertext)

print(plaintext.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


[Return to top](#toc)

# An ECB/CBC detection oracle<a name="prob11"></a>

In [9]:
class AESMode(Enum):
    ECB = auto()
    CBC = auto()
    
    def __str__(self):
        return f"{self.name.lower()}"

    
def generate_key(num_bytes: int=16) -> bytes:
    return bytes([random.randint(0,255) for x in range(num_bytes)])
generate_iv = generate_key


def aes_dice_roll() -> AESMode:
    dice_roll = random.randint(0, 1)
    if dice_roll == 0:
        return AESMode.ECB
    else:
        return AESMode.CBC

In [23]:
def encryption_oracle(plaintext: bytes, block_size: int=128) -> Tuple[List[str], bytes]:
    # pre- and appending random 5-10 bytes to plaintext
    plaintext = bytes([random.randint(0, 255) for x in range(random.randint(5, 10))]) + plaintext
    plaintext += bytes([random.randint(0, 255) for x in range(random.randint(5, 10))])
    
    plaintext = pkcs7(plaintext, block_size=block_size)
    
    ciphertext = b""
    ciphermode = []
    while len(plaintext) > 0:
        segment = plaintext[:block_size]
        plaintext = plaintext[block_size:]
        roll = aes_dice_roll()
        
        if roll == AESMode.ECB:
            ciphermode.append(str(roll))
            key = generate_key()
            encipher = AES.new(key, AES.MODE_ECB)
            ciphertext += encipher.encrypt(segment)
        elif roll == AESMode.CBC:
            ciphermode.append(str(roll))
            key = generate_key()
            iv = generate_iv()
            ciphertext += aes_cbc_encipher
            
    return (ciphermode, ciphertext)

[Return to top](#toc)

# Byte-at-a-time ECB decryption (Simple)<a name="prob12"></a>

In [62]:
KEY = generate_key()

In [63]:
def aes_128_ecb(plaintext: bytes) -> bytes:
    key = KEY
    block_size = len(key)
    magic_text = '''
        Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
        aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
        dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
        YnkK
    '''
    magic_bytes = base64.b64decode(magic_text)
    
    plaintext += magic_bytes
    
    plaintext = pkcs7(plaintext, block_size=block_size)
    ciphertext = b""
    
    # pre- and appending random 5-10 bytes to plaintext
#     plaintext = bytes([random.randint(0, 255) for x in range(random.randint(5, 10))]) + plaintext
#     plaintext += bytes([random.randint(0, 255) for x in range(random.randint(5, 10))])
     
    encipher = AES.new(key, AES.MODE_ECB)
    ciphertext = encipher.encrypt(plaintext)
            
    return ciphertext

## Step 1: "Discover" block size
We know it should be 16, which means that the ciphertext length should be a factor of 16 and as bytes are added the ciphertext length should jump by 16 bytes

In [64]:
plaintext = b""
for i in range(10):
    plaintext += b"A"
    ciphertext = aes_128_ecb(plaintext)
    print(f"{i} - Ciphertext length: {len(ciphertext)}")

0 - Ciphertext length: 144
1 - Ciphertext length: 144
2 - Ciphertext length: 144
3 - Ciphertext length: 144
4 - Ciphertext length: 144
5 - Ciphertext length: 160
6 - Ciphertext length: 160
7 - Ciphertext length: 160
8 - Ciphertext length: 160
9 - Ciphertext length: 160


## Step 2: "Detect" the function is using ECB
We're going to make use of code written for problem 8 to do this

In [65]:
def exists_repeated_bytes(data: bytes, num_of_bytes: int) -> bytes:
    for idx in range(len(data) - num_of_bytes):
        segment = data[idx: idx+num_of_bytes]
        if data[idx+1:].find(segment) > -1:
            return segment
    return None

In [66]:
plaintext = b"A" * 128
ciphertext = aes_128_ecb(plaintext)
repeated_bytes = exists_repeated_bytes(ciphertext, 16)

print(f"ciphertext: {binascii.hexlify(ciphertext).decode()}")
print(f"repeated bytes: {binascii.hexlify(repeated_bytes).decode()}")

ciphertext: 15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836ef15c969afa3e83ac4aca2054c307836efebe8c343028a550389fa6ff7b24c31a1721ecad4aed385df56fa7363290a6928128c1b8d69a0c3622895b1eb94d3c34d7eb078779c7d2adb079ecb3143f625ebfb1233aedbd126c1ca066a71f7de8450b806111d67fbad25aeee0a22b816efb91f882b03e08af9f253df6f0ce36ef7a73e99c8747071ae0dd2c75d6f1e3a9584902e13054c40864632d9fa85d09ca621
repeated bytes: 15c969afa3e83ac4aca2054c307836ef


## Step 3
Because we know we're in ECB and that the block size is 128 bits/16 bytes, we're going to craft an input that is exactly one byte short and make note of the output. Our understanding of PKCS\#7 dictates the end of the block is going to be padding to get the plaintext to a size that is a factor of the block length, which means the first byte  of the mystery message will be the last byte of the 15 byte input.

In [67]:
plaintext = b"A" * 15
ciphertext = aes_128_ecb(plaintext)
print(f"first block: {binascii.hexlify(ciphertext[:16])}")
print(f"remainder of the ciphertext: {binascii.hexlify(ciphertext[16:])}")

first block: b'c1db1001fa2d1078e870fd1d5a6c23eb'
remainder of the ciphertext: b'4e99b07911001387d46e406420b68b9703218cdae6c1ed7dea4f77ef3e632b82d54595bc1c45364e84c3f58a17a5ebaf7b774e44f093df2af4ffc6aa95da3362c082432920c181273cc450c21914ab011d386708c67cb1b4bc41f4a93c7e7e76ac055e25d138b0b61cf0fb744ba94dc1f049536eb9d75931a847fd6fb5bab7d34e4176d0310f3e4a7239221bfd4d0103'


## Step 4 and 5
We're going feed different strings of every possible last byte to the function (e.g. "AAAAAAAAAAAAAAAA", "AAAAAAAAAAAAAAAB", etc.), checking the first block of every invocation. When we get a match, we've decrypted the first byte of the mystery text. This is because ECB is deterministic and the same input will always result in the same output.

In [68]:
for i in range(0x7f):
    plaintext = (b"A" * 15) + bytes([i])
    test_ciphertext = aes_128_ecb(plaintext)
    if test_ciphertext[:16] == ciphertext[:16]:
        print(f"First decrypted byte: {bytes([i])}")
        break

First decrypted byte: b'R'


Testing to see that if the technique works if I shorten the control plaintext by 1 byte and append the first decrypted byte to the end of it.

In [69]:
plaintext = b"A" * 14
ciphertext = aes_128_ecb(plaintext)
for i in range(0x9, 0x7f):
    plaintext = (b"A" * 14) + bytes([ord("R"), i])
    test_ciphertext = aes_128_ecb(plaintext)
    if test_ciphertext[:16] == ciphertext[:16]:
        print(f"Second decrypted byte: {bytes([i])}")
        break

Second decrypted byte: b'o'


## Step 6
We're going to now repeat steps 3, 4, and 5 to decrypt the remainder of the ciphertext. But we're going to modify the process a little by expanding the number of test bytes to the full ciphertext length.

In [70]:
MAGIC_CIPHERTEXT = aes_128_ecb(b"")
MAGIC_CIPHERTEXT_SIZE = len(MAGIC_CIPHERTEXT)
# MAGIC_CIPHERTEXT_SIZE = 16

decrypted_text = b""
candidate_plaintext = b"A" * (MAGIC_CIPHERTEXT_SIZE - 1)
for i in range(MAGIC_CIPHERTEXT_SIZE):
    candidate_ciphertext = aes_128_ecb(candidate_plaintext)
    char_identified = False
    for char in range(0x7f):
        plaintext = candidate_plaintext + decrypted_text + bytes([char])
        ciphertext = aes_128_ecb(plaintext)
        if candidate_ciphertext[:MAGIC_CIPHERTEXT_SIZE] == ciphertext[:MAGIC_CIPHERTEXT_SIZE]:
            decrypted_text += bytes([char])
            candidate_plaintext = candidate_plaintext[1:]
            char_identified = True
            break
    if not char_identified:
        print(f"Decryption failed on byte {i}")
        print(f"Remaining bytes: {MAGIC_CIPHERTEXT[i:]}")
        break
print(f"Decryption result: {decrypted_text}") 

Decryption failed on byte 139
Remaining bytes: b'\x85\xd0\x9c\xa6!'
Decryption result: b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"


Not entirely sure why the last 5 bytes always fail to decrypt. Might be a side effect of my decryption algorithm that junk data is appended at the end.

# ECB cut-and-paste<a name="prob13"></a>

In [43]:
def sanitize(user_input: str) -> str:
    user_input = user_input.replace("&", f"%{ord('&'):02x}").replace("=", f"%{ord('='):02x}")
    return user_input
    

def cookie_args_parser(args: str) -> dict:
    kwargs = {}
    for pair in args.split("&"):
        k, v = pair.split("=")
        kwargs[k] = v
    return kwargs


def cookie_args_encoder(args: dict) -> str:
    output = []
    for k, v in args.items():
        output.append(f"{sanitize(k)}={sanitize(v)}")
    return "&".join(output)


def profile_for(user_email: str) -> str:
    profile = {
        'email': user_email,
        'uid': str(uuid.uuid4()),
        'role': 'user'
    }
    return cookie_args_encoder(profile)

In [34]:
test_args = "foo=bar&baz=qux&zap=zazzle"
cookie_args_parser(test_args)

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}

In [50]:
KEY = generate_key()

encipher = AES.new(KEY, AES.MODE_ECB)
decipher = AES.new(KEY, AES.MODE_ECB)

secret = pkcs7(bytes(profile_for("foo@bar.com").encode('utf8')), block_size=len(KEY))

ciphertext = encipher.encrypt(secret)
print(f"ciphertext: {binascii.hexlify(ciphertext).decode()}")

plaintext = decipher.decrypt(ciphertext)

# clean up padding
if plaintext[-1] != b"\x01" and plaintext[-1] == plaintext[-2]:
    plaintext = plaintext[:-1 * plaintext[-1]]

print(f"decrypted results: {cookie_args_parser(plaintext.decode())}")

ciphertext: 94360d935e4ccdb13b29c4f0958b87f1eb3c3e71a1d3f189b9f72300f121bbd9e1f4be95e67be39a6c34e8b536d00035ab4270522b8c70a7f52ed90a3eed24e6369342e7efaca6904e3360d584b413c2
decrypted results: {'email': 'foo@bar.com', 'uid': 'fb2f624d-bf68-4bfc-b02f-905ae3a889be', 'role': 'user'}


[Return to top](#toc)

# Byte-at-a-time ECB decryption (Harder)<a name="prob14"></a>

In [24]:
KEY = generate_key()

In [56]:
def aes_128_ecb(plaintext: bytes) -> bytes:
    key = KEY
    block_size = len(key)
    magic_text = '''
        Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
        aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
        dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
        YnkK
    '''
    magic_bytes = base64.b64decode(magic_text)
    plaintext += magic_bytes
    
    # pre- and appending random 5-10 bytes to plaintext
    plaintext = bytes([random.randint(0, 255) for x in range(random.randint(5, 10))]) + plaintext
#     plaintext += bytes([random.randint(0, 255) for x in range(random.randint(5, 10))])
    
    plaintext = pkcs7(plaintext, block_size=block_size)
     
    encipher = AES.new(key, AES.MODE_ECB)
    ciphertext = encipher.encrypt(plaintext)
            
    return ciphertext

[Return to top](#toc)

# PKCS\#7 padding validation<a name="prob15"></a>

In [2]:
def pkcs7_validation(data: bytes) -> bool:
    # check to see if last byte is 0x01
    if data[-1] == 1:
        return True
    
    if data[-1] != 1 and data[-1] == data[-2]:
        pad_value = data[-1]
        for i in range(1, pad_value):
            if data[-1 * i] != data[-1 * (i + 1)]:
                return False
        return True
    
    return False

In [11]:
assert pkcs7_validation(b"ICE ICE BABY\x04\x04\x04\x04")

In [13]:
assert pkcs7_validation(b"ICE ICE BABY\x05\x05\x05\x05") is False

In [14]:
assert pkcs7_validation(b"ICE ICE BABY\x01\x02\x03\x04") is False

In [15]:
assert pkcs7_validation(b"ICE ICE BABY") is False

In [3]:
assert pkcs7_validation(b"ICE ICE BABY\x01")

[Return to top](#toc)

# CBC bitflipping attacks<a name="prob16"></a>

In [16]:
KEY = generate_key()
IV = generate_key()

In [23]:
def aes_cbc_encrypt(user_input: str) -> bytes:
    key = KEY
    iv = IV
    prepend_str = "comment1=cooking%20MCs;userdata="
    append_str = ";comment2=%20like%20a%20pound%20of%20bacon"
    user_input = user_input.replace(";", "';'").replace("=", "'='")
    plaintext = bytes(f"{prepend_str}{user_input}{append_str}".encode("utf8"))
    plaintext = pkcs7(plaintext, block_size=len(key))
    
    return aes_cbc_encipher(plaintext, key, iv)


def aes_cbc_decrypt(ciphertext: bytes) -> bool:
    key = KEY
    iv = IV
    plaintext = aes_cbc_decipher(ciphertext, key, iv)
    
    if plaintext.find(b";admin=true;") != -1:
        return True
    else:
        return False

Sanity check to make sure that this fails as expected.

In [24]:
user_input = ";admin=true;"
ciphertext = aes_cbc_encrypt(user_input)
aes_cbc_decrypt(ciphertext)

False

We're going to fuzz every bit of the first block size to see what happens

In [26]:
user_input = ";admin=true;"
ciphertext = aes_cbc_encrypt(user_input)

fuzz_bytes = [0x80, 0x40, 0x20, 0x10, 0x8, 0x4, 0x2, 0x1]
for idx in range(len(KEY)):
    for bitflip in fuzz_bytes:
        ciphertext_list = list(ciphertext)
        ciphertext_list[idx] = ciphertext_list[idx] ^ bitflip
        ciphertext = bytes(ciphertext_list)
        if aes_cbc_decrypt(ciphertext):
            print(f"bitflip {bitflip:02x} in byte {idx} caused the decryption to be true")

It didn't work, but something tells me I misunderstood when the instructions said "quote out" the semicolon and equal signs from the user input

[Return to top](#toc)